In [1]:
#import main python libraries
import pandas as pd
import numpy as np
import time

import openweathermapy.core as owm
from citipy import citipy
from pprint import pprint

#import api keys
from config import api_key

In [4]:
#generate random coordinates
lats = np.random.uniform(-90,90, size=2500)
lngs = np.random.uniform(-180,180, size=2500)
coordinates = zip(lats,lngs)

#create empty list to hold unique cities
unique_cities = []

#generate unique cities list
for lat_lng in coordinates:
    city_data = citipy.nearest_city(lat_lng[0], lat_lng[1])
    city_and_country = city_data.city_name + ", " + city_data.country_code
    
    if city_and_country not in unique_cities:
        unique_cities.append(city_and_country)

num_unique_cities = len(unique_cities)
print(num_unique_cities)

1422


In [5]:
pprint(unique_cities)

['yanam, in',
 'richards bay, za',
 'belushya guba, ru',
 'prado, br',
 'rikitea, pf',
 'mataura, pf',
 'namibe, ao',
 'ushuaia, ar',
 'touros, br',
 'vao, nc',
 'atar, mr',
 'bunol, es',
 'bluff, nz',
 'aras, no',
 'vaitupu, wf',
 'bolungarvik, is',
 'atuona, pf',
 'butaritari, ki',
 'puerto del rosario, es',
 'cabo san lucas, mx',
 'dikson, ru',
 'bredasdorp, za',
 'lockhart, us',
 'sibolga, id',
 'zamora, es',
 'soyo, ao',
 'edd, er',
 'severo-kurilsk, ru',
 'leningradskiy, ru',
 'kudahuvadhoo, mv',
 'fortuna, us',
 'bethel, us',
 'sentyabrskiy, ru',
 'egvekinot, ru',
 'hermanus, za',
 'troitsko-pechorsk, ru',
 'brownwood, us',
 'nikolskoye, ru',
 'dzhusaly, kz',
 'lolua, tv',
 'kodiak, us',
 'avarua, ck',
 'the valley, ai',
 'cape town, za',
 'longyearbyen, sj',
 'kapaa, us',
 'kavaratti, in',
 'pisco, pe',
 'ambodifototra, mg',
 'shelburne, ca',
 'mareeba, au',
 'dharmanagar, in',
 'hilo, us',
 'jamestown, sh',
 'thompson, ca',
 'arraial do cabo, br',
 'barentsburg, sj',
 'mount g

In [17]:
#API call limit
calls_per_minute = 60

#Callsets gives us the number of times we have to loop through unique_cities to process every record, 
#while not exceeding API Usage limits
callsets = int(num_unique_cities/calls_per_minute) + 1

print(callsets)

24


In [11]:
#set callsets to 2 for debugging purposes
#callsets = 2

In [18]:
#settings for our Open Weather Map api calls
owm_settings = {"units": "imperial", "appid": api_key}

#Define list to log results for our OMW API Calls
api_call_log = []

#Create empty list that will track which cities from unique_cities successfully return OWM data
cities_in_owm = []

#For each callset, we will subset 60 cities from unique_cities into cities_to_process
cities_to_process = []

#Set initial indexes for subsetting into cities_to_process. We will increment by 60 each loop.
start = 0
end = 60

#start data retrieval    
begin_message = "----------------------------------\n" \
              + "Data Retrieval: Starting\n" \
              + "----------------------------------"

print(begin_message)

api_call_log.append(begin_message)

for callset in range(callsets):
    
    callset_group = "***********\n" \
                  + f"Beginning callset group: {callset}\n" \
                  + "***********"
    print(callset_group)
    
    api_call_log.append(callset_group)
    
    #get subset from unique_cities 
    cities_to_process = unique_cities[start:end]
    
    for i, city in enumerate(cities_to_process):

        try: 
            # test if city from cities_to_process is listed in open weather map. set to current_city if so.
            current_city = owm.get_current(city, **owm_settings)
            
            #pull out specific fields from current_city as a list, and then append list to cities_in_owm 
            cities_in_owm.append([current_city("id"),
                                  current_city("name"),
                                  current_city("sys.country"),
                                  current_city("coord.lat"),
                                  current_city("coord.lon"),
                                  current_city("main.temp_max"),
                                  current_city("main.humidity"),
                                  current_city("clouds.all"),
                                  current_city("wind.speed"),
                                  current_city("dt")])
            #log successful API response 
            owm_response = f"Processed Record {i} in callset group: {callset} | {city}"

        except:
            # if city from cities_to_process was NOT in open weather map, log unsuccessful API response
            owm_response = f"Skipping Record {i} in callset group: {callset} | {city} NOT FOUND"
        
        #print owm_response
        print(owm_response)
        
        #append response to api_call_log
        api_call_log.append(owm_response)
    
    #setup next set of cities to subset from unique_cities
    start = end
    end = end + 60
    
    #pause for 61 seconds to not exceed API call limits
    time.sleep(61)

#end data collection

#count cities_in_owm to get total number of successful API calls 
apicalls_succesful = len(cities_in_owm)

#subtract apicalls_succesful from num_unique_cities to get number of failed API calls
apicalls_failed = num_unique_cities - apicalls_succesful

end_message = "----------------------------------\n" \
            + "Data Retrieval: Complete\n"\
            + "----------------------------------\n"\
            + "----------------------------------\n"\
            + "Results \n"\
            + "----------------------------------\n"\
            + f"Callset groups: {callsets}\n"\
            + f"API Calls - Attempts: {num_unique_cities}\n"\
            + f"API Calls - Successful:{apicalls_succesful}\n"\
            + f"API Calls - Failed:{apicalls_failed}\n"

print(end_message)

api_call_log.append(end_message)


----------------------------------
Data Retrieval: Starting
----------------------------------
***********
Beginning callset group: 0
***********
Processed Record 0 in callset group: 0 | atuona, pf
Skipping Record 1 in callset group: 0 | mys shmidta, ru NOT FOUND
Processed Record 2 in callset group: 0 | trinidad, cu
Processed Record 3 in callset group: 0 | bredasdorp, za
Processed Record 4 in callset group: 0 | ushuaia, ar
Processed Record 5 in callset group: 0 | castro, cl
Processed Record 6 in callset group: 0 | haradok, by
Processed Record 7 in callset group: 0 | port alfred, za
Processed Record 8 in callset group: 0 | troitsko-pechorsk, ru
Processed Record 9 in callset group: 0 | mareeba, au
Processed Record 10 in callset group: 0 | bismarck, us
Processed Record 11 in callset group: 0 | ribeira grande, pt
Processed Record 12 in callset group: 0 | talnakh, ru
Processed Record 13 in callset group: 0 | pitsunda, ge
Processed Record 14 in callset group: 0 | punta arenas, cl
Processed R

Processed Record 46 in callset group: 4 | jonkoping, se
Skipping Record 47 in callset group: 4 | paradwip, in NOT FOUND
Processed Record 48 in callset group: 4 | sumter, us
Processed Record 49 in callset group: 4 | touros, br
Processed Record 50 in callset group: 4 | tapes, br
Processed Record 51 in callset group: 4 | nyurba, ru
Processed Record 52 in callset group: 4 | shiraoi, jp
Processed Record 53 in callset group: 4 | iskateley, ru
Processed Record 54 in callset group: 4 | taoudenni, ml
Processed Record 55 in callset group: 4 | talaya, ru
Processed Record 56 in callset group: 4 | tezu, in
Processed Record 57 in callset group: 4 | dhidhdhoo, mv
Processed Record 58 in callset group: 4 | coleraine, gb
Processed Record 59 in callset group: 4 | amapa, br
***********
Beginning callset group: 5
***********
Processed Record 0 in callset group: 5 | lewistown, us
Processed Record 1 in callset group: 5 | shebalino, ru
Processed Record 2 in callset group: 5 | clyde river, ca
Processed Record 

Processed Record 37 in callset group: 9 | esperance, au
Processed Record 38 in callset group: 9 | las vegas, us
Skipping Record 39 in callset group: 9 | kuytun, cn NOT FOUND
Processed Record 40 in callset group: 9 | zhaotong, cn
Processed Record 41 in callset group: 9 | trincomalee, lk
Processed Record 42 in callset group: 9 | waipawa, nz
Processed Record 43 in callset group: 9 | ikom, ng
Skipping Record 44 in callset group: 9 | marcona, pe NOT FOUND
Processed Record 45 in callset group: 9 | fayaoue, nc
Processed Record 46 in callset group: 9 | ngunguru, nz
Processed Record 47 in callset group: 9 | baghdad, iq
Processed Record 48 in callset group: 9 | yerofey pavlovich, ru
Processed Record 49 in callset group: 9 | albion, mu
Processed Record 50 in callset group: 9 | shingu, jp
Processed Record 51 in callset group: 9 | manosque, fr
Processed Record 52 in callset group: 9 | katobu, id
Processed Record 53 in callset group: 9 | caconda, ao
Processed Record 54 in callset group: 9 | sydney, 

***********
Beginning callset group: 12
***********
Skipping Record 0 in callset group: 12 | kristiinankaupunki, fi NOT FOUND
Skipping Record 1 in callset group: 12 | bargal, so NOT FOUND
Processed Record 2 in callset group: 12 | obo, cf
Processed Record 3 in callset group: 12 | carbondale, us
Processed Record 4 in callset group: 12 | kumbo, cm
Processed Record 5 in callset group: 12 | tofield, ca
Processed Record 6 in callset group: 12 | silistea crucii, ro
Processed Record 7 in callset group: 12 | kyra, ru
Skipping Record 8 in callset group: 12 | mazulskiy, ru NOT FOUND
Processed Record 9 in callset group: 12 | aldan, ru
Processed Record 10 in callset group: 12 | valence, fr
Skipping Record 11 in callset group: 12 | jiddah, sa NOT FOUND
Processed Record 12 in callset group: 12 | pulandian, cn
Processed Record 13 in callset group: 12 | sept-iles, ca
Processed Record 14 in callset group: 12 | poum, nc
Processed Record 15 in callset group: 12 | kaseda, jp
Processed Record 16 in callset 

Processed Record 22 in callset group: 14 | kenai, us
Processed Record 23 in callset group: 14 | constitucion, cl
Processed Record 24 in callset group: 14 | kandara, ke
Processed Record 25 in callset group: 14 | bitung, id
Processed Record 26 in callset group: 14 | galle, lk
Processed Record 27 in callset group: 14 | morogoro, tz
Processed Record 28 in callset group: 14 | limoeiro, br
Processed Record 29 in callset group: 14 | vernon, us
Processed Record 30 in callset group: 14 | soyo, ao
Processed Record 31 in callset group: 14 | vaitape, pf
Processed Record 32 in callset group: 14 | aloleng, ph
Skipping Record 33 in callset group: 14 | balimo, pg NOT FOUND
Processed Record 34 in callset group: 14 | ravar, ir
Processed Record 35 in callset group: 14 | biltine, td
Processed Record 36 in callset group: 14 | zenzeli, ru
Processed Record 37 in callset group: 14 | san, ml
Processed Record 38 in callset group: 14 | carahue, cl
Processed Record 39 in callset group: 14 | sinop, tr
Processed Re

Processed Record 45 in callset group: 16 | sretensk, ru
Processed Record 46 in callset group: 16 | sampit, id
Processed Record 47 in callset group: 16 | sinegorye, ru
Processed Record 48 in callset group: 16 | haapiti, pf
Processed Record 49 in callset group: 16 | ibicarai, br
Processed Record 50 in callset group: 16 | nador, ma
Skipping Record 51 in callset group: 16 | badaojiang, cn NOT FOUND
Processed Record 52 in callset group: 16 | tiarei, pf
Processed Record 53 in callset group: 16 | balabac, ph
Processed Record 54 in callset group: 16 | hokitika, nz
Processed Record 55 in callset group: 16 | beibei, cn
Skipping Record 56 in callset group: 16 | longlac, ca NOT FOUND
Processed Record 57 in callset group: 16 | shevchenkove, ua
Processed Record 58 in callset group: 16 | victoria, cl
Processed Record 59 in callset group: 16 | suraia, ro
***********
Beginning callset group: 17
***********
Processed Record 0 in callset group: 17 | guwahati, in
Processed Record 1 in callset group: 17 | 

Processed Record 8 in callset group: 19 | chuguyevka, ru
Processed Record 9 in callset group: 19 | christchurch, nz
Processed Record 10 in callset group: 19 | kyren, ru
Processed Record 11 in callset group: 19 | svetlaya, ru
Processed Record 12 in callset group: 19 | mayumba, ga
Processed Record 13 in callset group: 19 | krasnoyarka, ru
Processed Record 14 in callset group: 19 | lazo, ru
Processed Record 15 in callset group: 19 | chapais, ca
Processed Record 16 in callset group: 19 | palmerston, au
Processed Record 17 in callset group: 19 | bandarbeyla, so
Processed Record 18 in callset group: 19 | salta, ar
Processed Record 19 in callset group: 19 | chara, ru
Processed Record 20 in callset group: 19 | batticaloa, lk
Processed Record 21 in callset group: 19 | chabahar, ir
Processed Record 22 in callset group: 19 | north bend, us
Processed Record 23 in callset group: 19 | khorixas, na
Processed Record 24 in callset group: 19 | axim, gh
Processed Record 25 in callset group: 19 | korla, c

Skipping Record 34 in callset group: 21 | samana, do NOT FOUND
Skipping Record 35 in callset group: 21 | lephepe, bw NOT FOUND
Skipping Record 36 in callset group: 21 | tanjong tokong, my NOT FOUND
Processed Record 37 in callset group: 21 | khvorostyanka, ru
Processed Record 38 in callset group: 21 | prescott, us
Processed Record 39 in callset group: 21 | santa maria da vitoria, br
Processed Record 40 in callset group: 21 | barcelos, br
Skipping Record 41 in callset group: 21 | mouzakion, gr NOT FOUND
Processed Record 42 in callset group: 21 | denpasar, id
Processed Record 43 in callset group: 21 | moanda, ga
Processed Record 44 in callset group: 21 | kitimat, ca
Processed Record 45 in callset group: 21 | porbandar, in
Processed Record 46 in callset group: 21 | nanning, cn
Processed Record 47 in callset group: 21 | sylvan lake, ca
Processed Record 48 in callset group: 21 | moville, ie
Processed Record 49 in callset group: 21 | wesselsbron, za
Processed Record 50 in callset group: 21 | 

In [19]:
cities_df = pd.DataFrame(cities_in_owm, columns=["CityID","Name","Country","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed (mph)","Date_Time"])

In [20]:
cities_df

CityID             Name Country    Lat     Lng  Max Temp  Humdity  \
0     4020109           Atuona      PF  -9.80 -139.03     80.02       73   
1     3534915         Trinidad      CU  21.81  -79.98     73.40      100   
2     1015776       Bredasdorp      ZA -34.53   20.04     62.60       93   
3     3833367          Ushuaia      AR -54.81  -68.31     42.80       87   
4     3896218           Castro      CL -42.48  -73.76     50.00       93   
...       ...              ...     ...    ...     ...       ...      ...   
1239  2823533          Templin      DE  53.12   13.50     48.00       93   
1240   211647            Lodja      CD  -3.52   23.60     73.35       95   
1241    69559            Zabid      YE  14.20   44.50     62.94       39   
1242  3996234  Lazaro Cardenas      MX  28.39 -105.62     46.99       83   
1243  2036892           Hohhot      CN  40.81  111.65     46.40       24   

      Cloudiness  Wind Speed (mph)   Date_Time  
0             29             14.65  1573883089  
1             43              3.36  1573883089  
2             91             17.22  1573883089  
3             40              5.82  1573883089  
4             75              9.17  1573883089  
...          ...               ...         ...  
1239          75              9.17  1573884608  
1240         100              2.01  1573884608  
1241           0              5.06  1573884608  
1242           2              1.01  1573884608  
1243          20              2.24  1573884608  

[1244 rows x 10 columns]

In [21]:
cities_df.to_csv("data/cities_in_owm.csv")

In [22]:
api_call_log_df = pd.DataFrame(api_call_log, columns=["API Call Log"])
api_call_log_df.to_csv("data/api_results.csv", index=False, header=False)